In [5]:
using QEDFeynmanDiagrams
using QEDcore
using QEDprocesses
using ComputableDAGs

In [6]:
n = 4
proc = ScatteringProcess(
    (Electron(), Photon()),                                                         # incoming particles
    (Electron(), ntuple(_ -> Electron(), n)..., ntuple(_ -> Positron(), n)...),     # outgoing particles
    (SpinUp(), PolX()),                                                          # incoming particle spin/pols
    (SpinUp(), ntuple(_ -> SpinUp(), 2*n)...)                                     # outgoing particle spin/pols
) # trident: ke -> e (ep)^n, with one spin/pol combination
psp = PhaseSpacePoint(
    proc,
    PerturbativeQED(),
    PhasespaceDefinition(SphericalCoordinateSystem(), ElectronRestFrame()),
    tuple((rand(SFourMomentum) for _ in 1:number_incoming_particles(proc))...),
    tuple((rand(SFourMomentum) for _ in 1:number_outgoing_particles(proc))...)
)
println("Diagrams for the process: $(length(feynman_diagrams(proc)))")
graph = generate_DAG(proc)

Diagrams for the process: 2059200


Graph:
  Nodes: Total: 25027, QEDFeynmanDiagrams.ComputeTask_Pair: 3785, DataTask: 12849, 
         QEDFeynmanDiagrams.ComputeTask_Propagator: 660, QEDFeynmanDiagrams.ComputeTask_PropagatePairs: 660, QEDFeynmanDiagrams.ComputeTask_SpinPolCumulation: 1, 
         QEDFeynmanDiagrams.ComputeTask_Triple: 6400, QEDFeynmanDiagrams.ComputeTask_BaseState: 11, QEDFeynmanDiagrams.ComputeTask_CollectTriples: 1, 
         QEDFeynmanDiagrams.ComputeTask_CollectPairs: 660
  Edges: 51125
  Total Compute Effort: 0.0
  Total Data Transfer: 0.0
  Total Compute Intensity: 0.0


In [7]:
# this is necessary when using RuntimeGeneratedFunctions
using RuntimeGeneratedFunctions
RuntimeGeneratedFunctions.init(@__MODULE__)

func = get_compute_function(graph, proc, cpu_st(), @__MODULE__)
func(psp)

214.58184879606935

In [8]:
using BenchmarkTools
@benchmark func($psp)

BenchmarkTools.Trial: 3206 samples with 1 evaluation.
 Range (min … max):  1.376 ms …  41.138 ms  ┊ GC (min … max): 0.00% … 95.58%
 Time  (median):     1.466 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.556 ms ± 742.525 μs  ┊ GC (mean ± σ):  4.17% ±  8.96%

  ▃▆▇██▇▅▄▃▁                        ▁ ▁ ▁                     ▂
  ███████████▇▇▇▆▆▆▄▃▃▅▄▄▃▅▅▅▃▁▄▅▆▇████████▇▇▇▅▇▅▆▃▃▅▄▅▅▆▅▃▅▅ █
  1.38 ms      Histogram: log(frequency) by time      2.64 ms <

 Memory estimate: 2.73 MiB, allocs estimate: 17761.